# Build a Recommendation System for Purchase Data

In [1]:
#Libraries

#Data
import sqlalchemy as sql

#Data Science
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor, BaselineOnly
from surprise import KNNBasic,KNNWithMeans,KNNWithZScore,KNNBaseline
from surprise import SVD,SVDpp,NMF
from surprise import SlopeOne, CoClustering
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.accuracy import rmse, mae
from surprise import dump

#Model Tracking
import mlflow
from mlflow.tracking import MlflowClient
# TRACKING_URI = 'http://mlflow:5000'
# mlflow.set_tracking_uri(TRACKING_URI)

#Utils
import os
import configparser
from collections import defaultdict
import tempfile
import time
import json

In [2]:
# Enviroment Variables
outdata = '../data'
outmodels = '../models'
dbconnPath = './dbconn.properties'

# Set dbconnection variables
config = configparser.RawConfigParser()
config.read(dbconnPath)
params = config
db_host=params.get('CONN', 'host')
db_port=params.get('CONN', 'port')
db_user=params.get('CONN', 'user')
db_pwd=params.get('CONN', 'password')
db_name=params.get('CONN', 'database')

# Set connection string
connection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'

## Data Preparation

We have to:

1) Split each list of items in the products column into rows

2) Count the number of products bought by a user

In [3]:
# connect to database
engine = sql.create_engine(connection_str)
connection = engine.connect()

In [4]:
print(f'{db_name} contains {engine.table_names()[0]}, {engine.table_names()[1]} tables')
print(f'{db_name} contains {engine.table_names()[0]} table')

database contains CUSTOMERID, TRANSACTIONS tables
database contains CUSTOMERID table


In [5]:
# Convert tables in dataframes (Don't do that if you have a large dataset. Use just samples)
df_cus = pd.read_sql("select * from CUSTOMERID", connection)
df_trx = pd.read_sql("select * from TRANSACTIONS", connection)
connection.close()

In [6]:
print(df_cus.shape)
df_cus.head()

(1000, 2)


,id,customerId
0,1,1553
1,2,20400
2,3,19750
3,4,6334
4,5,27773


In [7]:
print(df_trx.shape)
df_trx.head()

(62483, 3)


,id,customerId,products
0,1,0,20
1,2,1,2|2|23|68|68|111|29|86|107|152
2,3,2,111|107|29|11|11|11|33|23
3,4,3,164|227
4,5,5,2|2


In [8]:
data_prep_1=pd.DataFrame(df_trx.products.str.split('|').tolist(), index=df_trx.customerId)\
.stack()\
.reset_index()\
.groupby(['customerId', 0])\
.agg({0: 'count'})\
.rename(columns={0: 'purchase_count'})\
.reset_index()\
.rename(columns={0: 'productId'})

In [9]:
print(data_prep_1.head())
print('*'*40)
print(data_prep_1.info())

  customerId productId  purchase_count
0          0         1               2
1          0        13               1
2          0       136               2
3          0       157               1
4          0        19               3
****************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133585 entries, 0 to 133584
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   customerId      133585 non-null  object
 1   productId       133585 non-null  object
 2   purchase_count  133585 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.1+ MB
None


In [10]:
#Let's normalize data in a way to have a rank
data_prep_2 = pd.pivot_table(data=data_prep_1, index='customerId', columns='productId', values='purchase_count', aggfunc='sum')
data_prep_3 = (data_prep_2 - data_prep_2.min())/(data_prep_2.max() - data_prep_2.min())
data_prep_4 = data_prep_3.reset_index().melt(id_vars=['customerId'], value_name='prod_ratings').dropna()
data_prep_4.index = np.arange(0, len(data_prep_4))
data_prep_4['prod_ratings'] = data_prep_4['prod_ratings'].apply(lambda x: int((round(x, 2))*100))

In [11]:
print(data_prep_4.sort_values(by=['customerId']).head())

       customerId productId  prod_ratings
1307            0         1            10
54157           0        20             0
49722           0       198             0
46478           0        19            14
116292          0        69            20


Then, we get **Triplet Representation** with each column representing user, item and the given rating respectively.
Notice the rating goes from 0–100 (with 100 being the most number of purchase for an item and 0 being 0 purchase count for that item).
It's a kind of preference for each user


## Data Exploration

### Rating Distributions

In [ ]:
df_plt1=data_prep_4['prod_ratings']
fig, ax = plt.subplots(figsize=(7, 3))
plt.hist(df_plt1)
ax.set_xlabel('Syntetic Product Ratings')
ax.set_ylabel('Frequency')
ax.set_title('Rating Distribution')
plt.show()

#### Rating Distribution by ProductId

In [ ]:
df_plt_2=data_prep_4.groupby('productId')['prod_ratings'].count().reset_index()[0:50]
df_plt_2
fig, ax = plt.subplots(figsize=(7, 3))
plt.bar(df_plt_2.productId, df_plt_2.prod_ratings, )
ax.set_xlabel('Number of Ratings per product')
ax.set_ylabel('Count')
ax.set_title('Rating Distribution by ProductId')
plt.show()

In [ ]:
df_plt_2.sort_values(by='prod_ratings', ascending=False)[0:10]

In [ ]:
df_plt_2=data_prep_4.groupby('productId')['prod_ratings'].count()
df_plt_2
fig, ax = plt.subplots(figsize=(7, 3))
plt.hist(df_plt_2)
ax.set_xlabel('Number of Ratings')
ax.set_ylabel('Count')
ax.set_title('Rating Distribution by ProductId')
plt.show()

#### Ratings Distribution By User

In [ ]:
df_plt_3=data_prep_4.groupby('customerId')['prod_ratings'].count().reset_index()[0:50]
fig, ax = plt.subplots(figsize=(7, 3))
plt.bar(df_plt_3.customerId, df_plt_3.prod_ratings)
ax.set_xlabel('Number of Ratings per Customer')
ax.set_ylabel('Count')
ax.set_title('Rating Distribution by customerId')
plt.show()

In [ ]:
df_plt_3.sort_values(by='prod_ratings', ascending=False)[0:10]

In [ ]:
df_plt_3=data_prep_4.groupby('customerId')['prod_ratings'].count()
fig, ax = plt.subplots(figsize=(7, 3))
plt.hist(df_plt_3)
ax.set_xlabel('Number of Ratings per Customer')
ax.set_ylabel('Frequency')
ax.set_title('Rating Distribution by customerId')
plt.show()

## Model Training and Evaluation

To do that, I use **Surprise** which is an easy-to-use Python scikit for recommender systems.

I follow the Getting Started (https://surprise.readthedocs.io/en/stable/getting_started.html#getting-started)

Reference: https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

### Settings and Utility Functions (mlflow_tracker, get_Iu, get_Ui)

In [12]:
# A reader is required with the rating_scale param
mindata = data_prep_4.prod_ratings.min()
maxdata = data_prep_4.prod_ratings.max()
reader = Reader(rating_scale=(mindata,maxdata))

# The columns must correspond to user id, item id and ratings (in that order)
data_prep_5 = data_prep_4.rename(columns={'customerId': 'userID', 'productId':'itemID', 'prod_ratings':'rating'})
data_prep_5.to_csv(os.path.join(outdata, 'sample_toscore.csv'))
data = Dataset.load_from_df(data_prep_5[['userID', 'itemID', 'rating']], reader)

#Create the experiment
mlflow.set_experiment('Purchase Recommended System')

#Create Minio Bucket for Artefact Storage
%run -t 00_set_bucket.ipynb

def mlflow_tracker(reader_data, algo, params, rundesc='myrun'):
        
    # Store Algo name
    algo_name = str(algo.__class__.__name__)

    with mlflow.start_run(run_name=algo_name) as run:

        # Store run_id and experiment_id
        run_id=run.info.run_uuid
        experiment_id=run.info.experiment_id
        
        #Create model instance
        redic=cross_validate(algo, reader_data, **params)
        #Create a dataframe of means
        recdf=pd.DataFrame.from_dict(redic).mean(axis=0)

        #Log params
        mlflow.log_params(params)
        mlflow.log_metric('test_time', recdf.iloc[3])
        mlflow.log_metric('test_rmse_mean', recdf.iloc[0])
        mlflow.log_metric('test_mae_mean', recdf.iloc[1])
        mlflow.log_metric('fit_time', recdf.iloc[2])

        # Set the notes for Runs

        MlflowClient().set_tag(run_id,
                               "mlflow.note.content",
                               rundesc)

    return (run_id, experiment_id)

BucketAlreadyOwnedByYou: BucketAlreadyOwnedByYou: message: Your previous request to create the named bucket succeeded and you already own it.

### Basic usage: Automatic cross-validation

#### Basic Algorithms

##### random_pred.NormalPredictor
Algorithm predicting a random rating based on the distribution of the training set, which is assumed to be normal.
##### baseline_only.BaselineOnly
Algorithm predicting the baseline estimate for given user and item.

#### KNN algorithms

##### knns.KNNBasic
A basic collaborative filtering algorithm
##### knns.KNNWithMeans
A basic collaborative filtering algorithm, taking into account the mean ratings of each user.
##### knns.KNNWithZScore
A basic collaborative filtering algorithm, taking into account taking into account the z-score normalization of each user.
##### knns.KNNBaseline
A basic collaborative filtering algorithm taking into account a baseline rating.

#### Matrix Factorization-based algorithms
##### matrix_factorization.SVD
The famous SVD algorithm, as popularized by Simon Funk during the Netflix Prize.When baselines are not used, this is equivalent to Probabilistic Matrix Factorization
##### matrix_factorization.SVDpp
The SVD++ algorithm, an extension of SVD taking into account implicit ratings.
##### matrix_factorization.NMF
A collaborative filtering algorithm based on Non-negative Matrix Factorization.

#### Other Algorithms
##### slope_one.SlopeOne
A simple yet accurate collaborative filtering algorithm.
##### co_clustering.CoClustering
A collaborative filtering algorithm based on co-clustering.



In [ ]:
params = {'measures': ['RMSE', 'MAE'], 'cv':3, 'verbose': False}
# algos = [BaselineOnly(), NormalPredictor(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), KNNBaseline(), SVD(), SVDpp(), SlopeOne(), CoClustering()]

algos = [BaselineOnly()]

for algo in algos:
    mlflow_tracker(data, algo, params)
    time.sleep(1)

**Comment:  Based on the Mlflow tracking service, the best model is BaselineOnly algorithm**

### Tuning the Best Model

In [22]:
def preparer(data):
    trainset, testset = train_test_split(data, test_size=0.25)
    return trainset, testset

def trainer(trainset, bsl_options):
    algo = BaselineOnly(bsl_options=bsl_options)
    model = algo.fit(trainset)
    return model

def predictor(model, testset):
    predictions = model.test(testset)
    return model, predictions

def tuner(data, bsl_options_grid, param_model):
    
    gs = GridSearchCV(BaselineOnly, bsl_options_grid, **param_model)
    gs.fit(data)
    
    history_tune=pd.DataFrame.from_dict(gs.cv_results)
    
    best_bsl_options = gs.best_params['rmse']
    
    train, test = preparer(data)
    tuned_model = trainer(train, best_bsl_options)
    
    return param_model, history_tune, tuned_model

 # Define utils functions for prediction readability
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0

def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0

def mlflow_tune_tracker(data, algo_name, param_grid, param_model):

    with mlflow.start_run(run_name=algo_name) as run:
        
        # Store run_id and experiment_id
        run_id=run.info.run_uuid
        experiment_id=run.info.experiment_id
        
        #Train the model
        trainset, testset = preparer(data)
        
        #Tune
        params, history_tune, tuned_model = tuner(data, param_grid, param_model)

        #History
        for index, row in history_tune.iterrows():
            with mlflow.start_run(experiment_id=experiment_id, run_name=algo_name + str(index), nested=True) as subruns:
                
                bsl_options = row['params']
                params_tune = {**params, **bsl_options}

                #Log params
                mlflow.log_params(params_tune)
                mlflow.log_metric('fit_time',round(row['mean_fit_time'], 3))
                mlflow.log_metric('test_time', round(row['mean_test_time'], 3))
                mlflow.log_metric('test_rmse_mean', round(row['mean_test_rmse'], 3))
                mlflow.log_metric('test_mae_mean', round(row['mean_test_mae'], 3))
                
                #Log Model (artefact)
                temp = tempfile.NamedTemporaryFile(prefix="model", suffix=".pkl")
                temp_name = temp.name
                try:
                    model, predictions = predictor(trainer(trainset, bsl_options), testset)
                    dump.dump(temp_name, predictions, model)
                    mlflow.log_artifact(temp_name, 'model')
                finally:
                    temp.close()
                    
                 #Log best predictions
                df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
                df['Iu'] = df.uid.apply(get_Iu(trainset))
                df['Ui'] = df.iid.apply(get_Ui(trainset))
                df['err'] = abs(df.est - df.rui)
                best_predictions = df.sort_values(by='err')[:10]
                temp = tempfile.NamedTemporaryFile(prefix="best-predicitions", suffix=".csv")
                temp_name = temp.name
                try:
                    best_predictions.to_csv(temp_name, index=False)
                    mlflow.log_artifact(temp_name)
                finally:
                    temp.close()
        
    

In [23]:
param_grid = {'bsl_options' : {'method': ['als', 'sgd'],
              'n_epochs': [5, 15],
              'reg_u': [10, 20],
              'reg_i': [5,15]               
                }
              }

param_model = {
          'measures': ['RMSE', 'MAE'], 
          'cv':3
            }

# params, history_tune, tuned_model = tuner(data, param_grid, param_model)

# mlflow_tune_tracker(params, history_tune, tuned_model)

mlflow_tune_tracker(data, 'BaselineOnly', param_grid, param_model)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimati

TypeError: get_Iu() missing 1 required positional argument: 'uid'

In [ ]:
def mlflow_tune_tracker(tuned_model, params_tune, history_tune):
    
            trainset, testset = train_test_split(reader_data, test_size=testsize)
            algo_best = BaselineOnly(bsl_options=model_params)
            predictions = algo_best.fit(trainset).test(testset)
            
            #Log params
            mlflow.log_params(params)
            mlflow.log_metric('test_rmse_mean', round(rmse(predictions), 3))
            mlflow.log_metric('test_mae_mean', round(mae(predictions), 3))
            
            #Log Model (artefact)
            dump.dump(os.path.join(outputdir, 'model.pkl'), predictions, algo_best)
            mlflow.log_artifact(os.path.join(outputdir, 'model.pkl'), 'model')
            
            #Log best predictions
            df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
            df['Iu'] = df.uid.apply(get_Iu)
            df['Ui'] = df.iid.apply(get_Ui)
            df['err'] = abs(df.est - df.rui)
            best_predictions = df.sort_values(by='err')[:10]
            temp = tempfile.NamedTemporaryFile(prefix="best-predicitions", suffix=".csv")
            temp_name = temp.name
            try:
                best_predictions.to_csv(temp_name, index=False)
                mlflow.log_artifact(temp_name)
            finally:
                temp.close() # Delete the temp file
    

**This is the best parameter combination**

Then the best model is BaselineOnly with 
- method: 'als'
- n_epochs: 10
- reg_u: 12
- reg_i: 5

## Model validation

### Train, test and store

In [ ]:
trainset, testset = train_test_split(data, test_size=0.25)
algo_best = BaselineOnly(bsl_options=bsl_options)
predictions = algo_best.fit(trainset).test(testset)
print(round(rmse(predictions), 3))

### Load and predict

In [ ]:
predictions, algo = dump.load(outmodels)

### Single value

In [ ]:
test_sample = data_prep_4.head()
test_sample.head()

In [ ]:
uid = str(1007)  # raw user id (as in the ratings file). They are **strings**!
iid = str(0)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo_best.predict(uid, iid, r_ui=13, verbose=True)

### Score the entire dataset

In [ ]:
data_prep_5['predictions'] = data_prep_5.apply(lambda row:algo_best.predict(row['userID'], 
                     row['itemID'], row['rating']), axis = 1)
data_prep_5.head()
# predictions=list(data_prep_5['predictions'])

### Top 10 Recommended Items


In [ ]:
def get_top_n(predictions, n=10):
    
    '''Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

#     # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def get_top_n_ui(top, ui):
    try:
        return {k:v for k,v in top.items() if ui==k}
    except ValueError: # user was not part of the trainset
        return 0

In [ ]:
print(get_top_n_ui(get_top_n(predictions, n=10), '20400'))

## Register the model with the MLflow Model Registry API

In [ ]:
run_id, experiment_id = mlflow_tracker (data, algo, 'Trained', bsl_options, params, testsize=0.25, outputdir=outmodels, runame='run1', expdesc='myexperiment', rundesc='myrun')

### Model Registration

In [ ]:
model_name = 'Champion'
artifact_path = "model"
model_uri = "runs:/{run_id}/{artifact_path}".format(run_id=run_id, artifact_path=artifact_path)

model_details = mlflow.register_model(model_uri=model_uri, name=model_name)

In [ ]:
client = MlflowClient()
client.update_registered_model(
  name=model_details.name,
  description="This model provides recommendation for specific users and items based on purchase data. The data consists of user transactions"
)

client.update_model_version(
  name=model_details.name,
  version=model_details.version,
  description="This model was built with Surprise library. It is a ALS based BaselineOnly algorithm"
)